In [3]:
import os
import pickle
import shutil 
import meep as mp
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
import math
from IPython import embed

Using MPI version 4.0, 1 processes


hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


In [4]:
# set up simulation constants

rresolution =30

n_air = 1.0000003
n_Si = 1.45
n_SiO2 = 3.48

a = 0.5   # lattice period 

#pml_thickness = round(wavelength / 2,3)
pml_thickness = 0.780
height_hole = 5
height_SiO2 = height_hole
height_air = 7+2*pml_thickness
height_Si_initial= 0.5*(height_air-height_hole)
height_Si_add = 0.75*0.5*height_SiO2


cell_x = a
cell_y = a
cell_z = height_air

z_air = height_air

center_air = 0 
center_SiO2 = 0 
center_hole = 0
center_Si_initial = -0.5*height_hole -0.25*(height_air-height_hole)+0.005
center_Si_add = 0.5*height_Si_add

In [5]:

#initial_geometry setup

cell_size = mp.Vector3(cell_x,cell_y,cell_z)
pml_layers = [mp.PML(thickness = pml_thickness, direction = mp.Z)]

geometry = [mp.Block(size=mp.Vector3(mp.inf,mp.inf,height_air), 
                    center=mp.Vector3(0,0,center_air),
                    material=mp.Medium(index=n_air)),
           mp.Block(size=mp.Vector3(mp.inf,mp.inf,height_Si_initial),
                   center=mp.Vector3(0,0,center_Si_initial),
                   material=mp.Medium(index=n_Si))]


k_point = mp.Vector3(0,0,0)

## source ##
center_source = -0.5*height_hole-0.125*(height_air-height_hole) 
source_cmpt = mp.Ey

wavelength = 1.55
freq = 1 / wavelength


sources = [mp.Source(mp.ContinuousSource(frequency=freq),
                    component=source_cmpt,
                    center=mp.Vector3(0,0,center_source),
                    size=mp.Vector3(cell_x,cell_y,0))]


## flux monitor ##
nfreq = 1
df = 0
fr_center = 0.5*height_hole+0.125*(height_air-height_hole)
fr = mp.FluxRegion(center=mp.Vector3(0,0,fr_center), 
            size=mp.Vector3(cell_x, cell_y, 0))



if source_cmpt == mp.Ey:
    symmetries = [mp.Mirror(mp.X, phase=+1), #epsilon has mirror symmetry in x and y, phase doesn't matter
                  mp.Mirror(mp.Y, phase=-1)] #but sources have -1 phase when reflected normal to their direction
elif src_cmpt == mp.Ex:                      #use of symmetries important here, significantly speeds up sim
    symmetries = [mp.Mirror(mp.X, phase=-1),
                  mp.Mirror(mp.Y, phase=+1)]
elif src_cmpt == mp.Ez:
    symmetries = [mp.Mirror(mp.X, phase=+1),
                  mp.Mirror(mp.Y, phase=+1)]




In [6]:
sim = mp.Simulation(cell_size=cell_size,
                    geometry=geometry,
                    sources=sources,
                    k_point=k_point,
                    boundary_layers=pml_layers,
                    symmetries=symmetries,
                    resolution=resolution)
flux_object = sim.add_flux(freq, df, nfreq, fr)


sim.run(until=200)


combined_initial_flux = mp.get_fluxes(flux_object)[0]



# In[4]:


geometry.append(mp.Block(size=mp.Vector3(mp.inf,mp.inf,height_SiO2), 
                    center=mp.Vector3(0,0,center_SiO2),
                    material=mp.Medium(index=n_SiO2)))

geometry.append(mp.Block(size=mp.Vector3(mp.inf,mp.inf,height_Si_add), 
                    center=mp.Vector3(0,0,center_Si_add),
                    material=mp.Medium(index=n_Si)))


## initialize matrix for data collection ##
##########################################

num_holes = 15  # this is the number of pillars we will build
num_height = 15
min_r = 0.1
max_r = 0.245
min_h = 1.02
max_h = 5
data = np.zeros((4,num_holes))






NameError: name 'resolution' is not defined

In [5]:
sim.reset_meep()

combined_data = {}
height_list = []

def run_sim(radius,height,sim) :
    print(radius)
    
    geometry.append(mp.Cylinder(radius=radius,
                        height=height,
                        axis=mp.Vector3(0,0,1),
                        center=mp.Vector3(0,0,center_hole),
                        material=mp.Medium(index=n_air)))
                
                
    sim = mp.Simulation(cell_size=cell_size,
                        geometry=geometry,
                        sources=sources,
                        k_point=k_point,
                        boundary_layers=pml_layers,
                        symmetries=symmetries,
                        resolution=resolution)
                    #save_geometry(sim, radius, height)
                        
    flux_object = sim.add_flux(freq, df, nfreq, fr)  
    sim.run(until=200)
    res = sim.get_eigenmode_coefficients(flux_object, [1], eig_parity=mp.ODD_Y)
    coeffs = res.alpha

    flux = abs(coeffs[0,0,0]**2)
    phase = np.angle(coeffs[0,0,0]) 

    if phase<-np.pi:
        phase = phase +2*np.pi
    elif phase>-np.pi and phase<np.pi:
        phase = phase
    else:
        phase = phase - 2*np.pi
                                                    
   #radii_list.append(radius)
   # flux_list.append(flux)
   # phase_list.append(phase)
   # new_heightlist.append(height)
                                                                        
    
    sim.reset_meep()
    geometry.pop(-1)

    return phase, flux

def save_geometry(sim, radius, height):
    fig = plt.figure(figsize=(10,10))
    sim.plot2D(output_plane=plot_plane)

pbar = tqdm(total=num_height*num_holes,leave=False)
new_heightlist =[]
for h, height in enumerate(np.linspace(min_h,max_h,num_height)):
    radii_list=[] 
    flux_list, phase_list=[],[]
    new_heightlist.append(height)
    for r, radius in enumerate(np.linspace(min_r,max_r,num_holes)):
        radii_list.append(radius)
        phase, flux=run_sim(radius,height,sim)
        phase_list.append(phase)
        flux_list.append(flux)
        
        print(h,r)
        print("#########################################")
        print(f"phase: {phase} flux: {flux} radis:{radius} depth:{height}")
        pbar.update(1)
    combined_data[h] = {
            'height' : height,
            'flux' : flux_list,
            'phase' : phase_list,
            'radii' : radii_list,
            'new_height' : new_heightlist 
             }
       
pbar.close()

-----------
Initializing structure...
Halving computational cell along direction x
Halving computational cell along direction y
time for choose_chunkdivision = 0.000627995 s
Working in 3D dimensions.
Computational cell is 0.5 x 0.5 x 7.65 with resolution 20
     block, center = (0,0,-0.15)
          size (1e+20,1e+20,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (0,0,-0.15)
          size (1e+20,1e+20,5.78)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 0.0315959 s
-----------


FloatProgress(value=0.0, description='0% done ', max=200.0)

run 0 finished at t = 200.0 (8000 timesteps)


  0%|                                                                                                                                                                               | 0/15 [00:00<?, ?it/s]Warning: grid volume is not an integer number of pixels; cell size will be rounded to nearest pixel.


0.0
-----------
Initializing structure...
Halving computational cell along direction x
Halving computational cell along direction y
time for choose_chunkdivision = 0.00105095 s
Working in 3D dimensions.
Computational cell is 0.5 x 0.5 x 7.65 with resolution 20
     block, center = (0,0,-0.15)
          size (1e+20,1e+20,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.1025,2.1025,2.1025)
     block, center = (0,0,-0.15)
          size (1e+20,1e+20,5.78)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.1025,2.1025,2.1025)
     cylinder, center = (0,0,2.23)
          radius 0, height 1.02, axis (0, 0, 1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 0.0302 s
-----------


...

FloatProgress(value=0.0, description='0% done ', max=200.0)

run 0 finished at t = 200.0 (8000 timesteps)
MPB solved for frequency_1(0,0,0.645161) = 0.645161 after 6 iters
Dominant planewave for band 1: (0.000000,0.000000,0.645161)


0.245
-----------
Initializing structure...
Halving computational cell along direction x
Halving computational cell along direction y
time for choose_chunkdivision = 0.00097394 s
Working in 3D dimensions.
Computational cell is 0.5 x 0.5 x 7.65 with resolution 20
     block, center = (0,0,-0.15)
          size (1e+20,1e+20,5)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.1025,2.1025,2.1025)
     block, center = (0,0,-0.15)
          size (1e+20,1e+20,5.78)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.1025,2.1025,2.1025)
     cylinder, center = (0,0,0.24)
          radius 0.245, height 5, axis (0, 0, 1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 0.0928838 s
-----------


FloatProgress(value=0.0, description='0% done ', max=200.0)

run 0 finished at t = 200.0 (8000 timesteps)
MPB solved for frequency_1(0,0,0.645161) = 0.645161 after 6 iters
Dominant planewave for band 1: (0.000000,0.000000,0.645161)


In [11]:
import pickle
combined_data = pickle.load(open("/develop/results/combined_data.pkl", "rb"))

In [12]:
# print(combined_data)
# print(combined_initial_flux)

In [34]:
## pickle operations

for k,j in enumerate(np.linspace(0,num_height-1,num_height)):
    for v,l in enumerate(np.linspace(0,num_holes-1, num_holes)):
        x=combined_data[k]['radii']
        y=combined_data[k]['new_height']
        z=combined_data[k]['phase']
        z_1=combined_data[k]['flux']

In [70]:
transmission = []
x = []
y = []
phases = []


for k,j in enumerate(np.linspace(0,num_height-1,15)):
    y.append(combined_data[k]['height'])
    # combined_data[][]
    x.append(combined_data[k]['radii'])
    #y.append(combined_data[k]['new_height'])
    phases.append(combined_data[k]['phase'])
    transmission.append(combined_data[k]['flux'])

    
#x = x[0]
x = np.asarray(x[0])
y = np.asarray(y)
phases = np.asarray(phases)
transmission = np.asarray(transmission)

In [13]:
import seaborn as sns
import csv
import pandas as pd

In [14]:
def calculate_average(numbers):
    total = sum(numbers)
    count = len(numbers)
    average = total / count
    return average

for key in combined_data:
    data = combined[key]['flux']
    radius_data[key]['flux'] = calculate_average(data)

NameError: name 'combined' is not defined

In [2]:
df = pd.read_csv('height_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'height_data.csv'

In [ ]:
sns.set_theme(style="ticks")

# Load the dataset and initialize the figure
g = sns.JointGrid(data=df, x="height", y="flux", marginal_ticks=True)

# y axis
g.ax_joint.set(yscale="linear")

# Create an inset legend for the histogram colorbar
cax = g.figure.add_axes([.15, .55, .02, .2])

# Add the joint and marginal histogram plots
g.plot_joint(
    sns.histplot, discrete=(True, False),
    cmap="light:#03012d", pmax=.8, cbar=True, cbar_ax=cax
)
g.plot_marginals(sns.histplot, element="step", color="#03012d")

In [ ]:
sns.set_theme(style="ticks")

# Load the dataset and initialize the figure
g = sns.JointGrid(data=df, x="height", y="phase", marginal_ticks=True)

# y axis
g.ax_joint.set(yscale="linear")

# Create an inset legend for the histogram colorbar
cax = g.figure.add_axes([.15, .55, .02, .2])

# Add the joint and marginal histogram plots
g.plot_joint(
    sns.histplot, discrete=(True, False),
    cmap="light:#03012d", pmax=.8, cbar=True, cbar_ax=cax
)
g.plot_marginals(sns.histplot, element="step", color="#03012d")